In [31]:
from function import *
import numpy as np
import networkx as nx
import time
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt

#### LOAD THE DATA

In [32]:
# Using Random Data
# N = 100000                # Number of data
# d = 2                     # Number of features
# precision = 2             # Control precision bit of the output
# dim = d * (2 * precision) # Size of the QUBO matrix
# degree = 2                # Degree of the polynomial regression

# # Generate Random Data
# X = np.random.rand(N, d)
# Y = np.random.rand(N)

# Using Dataset
start_time = time.time()

dataset_path = 'dataset/processed dataset/dataset_6.csv'
dataset_name = dataset_path.split('/')[-1]
df = pd.read_csv(dataset_path)

X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
d = X.shape[1]
precision = 3
dim = d * (2 * precision)
degree = 4

# Train-Test data splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# If you want to perform polynomial regression
X_train, d, dim = polynomialForm(X_train, d, dim, precision, degree)
X_test, _, _ = polynomialForm(X_test, d, dim, precision, degree) 

end_time = time.time()
preproccesing1_time = end_time - start_time

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(29386, 35)
(7347, 35)
(29386,)
(7347,)


#### COMPUTING SOME IMPORTANT VARIABLES
Initiating variables that required to creating QUBO Matrix

In [33]:
start_time = time.time()

XtX = np.matmul(X_train.T, X_train) # Covariance matrix
XtY = np.matmul(X_train.T, Y_train) # Relation between Features and Y

end_time = time.time()
preproccesing2_time = end_time - start_time

print(XtX)
print(XtY)

[[29386.         16240.93559752 15740.82719212 ...  4748.5355734
   6657.27322622 10287.08274577]
 [16240.93559752  9839.55017915  8441.65442503 ...  2308.85304122
   3285.0310112   5142.46091234]
 [15740.82719212  8441.65442503  8904.55036801 ...  2814.87266279
   3742.88931846  5439.91433264]
 ...
 [ 4748.5355734   2308.85304122  2814.87266279 ...  1102.60693007
   1487.24298553  2156.84788926]
 [ 6657.27322622  3285.0310112   3742.88931846 ...  1487.24298553
   2156.84788926  3371.29197501]
 [10287.08274577  5142.46091234  5439.91433264 ...  2156.84788926
   3371.29197501  5712.47370927]]
[12370.8940614   6751.61819718  6712.30893073  8604.08434378
  4078.83226244  3537.41697698  4492.56638704  3848.74957158
  4666.15065447  6435.22677945  2656.50401791  2081.61407695
  2588.65781347  1947.77399935  2357.22842287  3258.64856273
  2318.54320802  2676.62069999  3479.26393627  5055.1772261
  1828.77031963  1329.50744872  1609.39721586  1110.05413573
  1323.67452725  1813.73658755  1122

In [34]:
total_preproccesing_time = preproccesing1_time + preproccesing2_time

#### FIXSTAR AMPLIFY SIMULATED ANNEALER

In [35]:
start_time = time.time()
model = generateQuboMatrix_fixstar(XtX, XtY, dim, precision, d)
end_time = time.time()
fixstar_preproccesing_time = end_time - start_time + total_preproccesing_time

start_time = time.time()
sampleset_fixstar = sampling_fixstar(model)
end_time = time.time()
fixstar_execution_time = end_time - start_time

start_time = time.time()
distributions_fixstar, weights_fixstar = solve_fixstar(sampleset_fixstar, dim, precision, d)
end_time = time.time()
fixstar_postproccesing_time = end_time - start_time

In [36]:
Y_train_pred_fixstar = np.dot(X_train, weights_fixstar)
Y_test_pred_fixstar = np.dot(X_test, weights_fixstar)

# Evaluate on training data
train_mse_fixstar = mean_squared_error(Y_train, Y_train_pred_fixstar)
train_mae_fixstar = mean_absolute_error(Y_train, Y_train_pred_fixstar)
train_r2_fixstar = r2_score(Y_train, Y_train_pred_fixstar) 

# Evaluate on test data
test_mse_fixstar = mean_squared_error(Y_test, Y_test_pred_fixstar)
test_mae_fixstar = mean_absolute_error(Y_test, Y_test_pred_fixstar)
test_r2_fixstar = r2_score(Y_test, Y_test_pred_fixstar)

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_fixstar:.5f}, MAE: {train_mae_fixstar:.5f}, R²: {train_r2_fixstar:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_fixstar:.5f}, MAE: {test_mae_fixstar:.5f}, R²: {test_r2_fixstar:.5f}")  

Fixstar Training Evaluation:
MSE: 0.03583, MAE: 0.14857, R²: 0.07239

Fixstar Testing Evaluation:
MSE: 0.03573, MAE: 0.14798, R²: 0.07375


#### NEAL SIMULATED ANNEALER

In [37]:
start_time = time.time()
Q = generateQuboMatrix_neal(XtX, XtY, precision, d)
end_time = time.time()
neal_preproccesing_time = end_time - start_time + total_preproccesing_time

start_time = time.time()
sampleset_neal = sampling_neal(Q)
end_time = time.time()
neal_execution_time = end_time - start_time 

start_time = time.time()
distributions_neal, weights_neal = solve_neal(sampleset_neal, dim, precision, d)
end_time = time.time()
neal_postproccesing_time = end_time - start_time 

In [38]:
Y_train_pred_neal = np.dot(X_train, weights_neal)
Y_test_pred_neal = np.dot(X_test, weights_neal)

# Evaluate on training data
train_mse_neal = mean_squared_error(Y_train, Y_train_pred_neal)
train_mae_neal = mean_absolute_error(Y_train, Y_train_pred_neal)
train_r2_neal = r2_score(Y_train, Y_train_pred_neal)

# Evaluate on test data
test_mse_neal = mean_squared_error(Y_test, Y_test_pred_neal)
test_mae_neal = mean_absolute_error(Y_test, Y_test_pred_neal)
test_r2_neal = r2_score(Y_test, Y_test_pred_neal)

# Print the metrics
print("Neal Training Evaluation:")
print(f"MSE: {train_mse_neal:.5f}, MAE: {train_mae_neal:.5f}, R²: {train_r2_neal:.5f}")

print("\nNeal Testing Evaluation:")
print(f"MSE: {test_mse_neal:.5f}, MAE: {test_mae_neal:.5f}, R²: {test_r2_neal:.5f}")

Neal Training Evaluation:
MSE: 0.04139, MAE: 0.16127, R²: -0.07174

Neal Testing Evaluation:
MSE: 0.04097, MAE: 0.15922, R²: -0.06196


#### SCIKIT LEARN (TRADISIONAL WAY)

In [39]:
start_time = time.time() 

clf = LinearRegression()
clf.fit(X_train, Y_train)
print("Coefficients:", clf.coef_)

Y_train_pred_sklearn = clf.predict(X_train)
Y_test_pred_sklearn = clf.predict(X_test)

# Evaluate on training data
train_mse_sklearn = mean_squared_error(Y_train, Y_train_pred_sklearn)
train_mae_sklearn = mean_absolute_error(Y_train, Y_train_pred_sklearn)
train_r2_sklearn = r2_score(Y_train, Y_train_pred_sklearn) 

# Evaluate on test data
test_mse_sklearn = mean_squared_error(Y_test, Y_test_pred_sklearn)
test_mae_sklearn = mean_absolute_error(Y_test, Y_test_pred_sklearn)
test_r2_sklearn = r2_score(Y_test, Y_test_pred_sklearn)

end_time = time.time()  
sklearn_execution_time = end_time - start_time

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_sklearn:.5f}, MAE: {train_mae_sklearn:.5f}, R²: {train_r2_sklearn:.5f}")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_sklearn:.5f}, MAE: {test_mae_sklearn:.5f}, R²: {test_r2_sklearn:.5f}")  

Coefficients: [ 6.90535756e-13  2.43077632e+01  1.74794574e+01  9.77929268e-01
 -5.28776706e+01 -5.36306701e+01 -1.73098515e+00 -2.75883308e+01
 -4.85099290e+00 -2.49131882e+00  4.64934102e+01  6.46823799e+01
  8.84037032e+00  4.84549501e+01  2.59034445e+00 -1.19181831e-01
  2.40697343e+01 -5.65777627e-01  8.01330010e+00  1.27181337e+00
 -1.38599705e+01 -2.62241584e+01 -5.70745544e+00 -2.21363308e+01
 -1.00107123e+01  3.17960255e+00 -2.21770658e+01  8.09512200e+00
 -1.44015104e+00 -3.00126488e+00 -8.61277261e+00  4.21123067e+00
 -7.30637034e+00 -7.92830851e-01  4.47695577e-01]
Fixstar Training Evaluation:
MSE: 0.03545, MAE: 0.14747, R²: 0.08223

Fixstar Testing Evaluation:
MSE: 0.03537, MAE: 0.14693, R²: 0.08315


#### VISUALIZATION OF GRAPH

In [40]:
# # Initialize the graph and add all nodes first
# G = nx.Graph()
# nodes = set()
# for (i, j) in Q.keys():
#     nodes.add(i)
#     nodes.add(j)
# G.add_nodes_from(nodes)

# # Now add edges and node attributes
# for (i, j), weight in Q.items():
#     if weight != 0 and i <= j:  # Avoid duplicates and zero-weights
#         if i == j:
#             G.nodes[i]['bias'] = weight  # Now safe because node exists
#         else:
#             G.add_edge(i, j, weight=weight)

# # Visualization
# plt.figure(figsize=(12, 10))
# pos = nx.circular_layout(G)

# # Draw nodes and edges
# nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=800)
# nx.draw_networkx_labels(G, pos, font_size=12)

# # Scale edge widths by absolute weight
# max_weight = max(abs(d['weight']) for _, _, d in G.edges(data=True))
# edge_widths = [0.1 + 2 * abs(G[u][v]['weight']) / max_weight for u, v in G.edges()]
# nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color='gray')

# # Add edge labels (rounded)
# edge_labels = {(u, v): f"{d['weight']:.1f}" for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# # Add node biases
# for node, (x, y) in pos.items():
#     bias = G.nodes[node].get('bias', 0)
#     plt.text(x, y-0.1, f"bias={bias:.1f}", ha='center', fontsize=8, color='red')

# plt.title("QUBO Graph (Edge Width ∝ |Weight|)", fontsize=14)
# plt.axis('off')
# plt.show()

In [41]:
# pos = nx.spring_layout(G, k=0.5, weight='weight', iterations=100)
# nx.draw(G, pos, with_labels=True, 
#        node_color='lightblue', node_size=800,
#        width=[0.1 + 2*abs(G[u][v]['weight'])/max_weight for u,v in G.edges()],
#        edge_color='gray')

In [42]:
# from mpl_toolkits.mplot3d import Axes3D

# pos = nx.spring_layout(G, dim=3, weight='weight')
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for u, v in G.edges():
#     ax.plot([pos[u][0], pos[v][0]], 
#             [pos[u][1], pos[v][1]], 
#             [pos[u][2], pos[v][2]], 
#             'gray')
# ax.scatter(*np.array(list(pos.values())).T, s=100, c='lightblue')
# plt.show()

#### FINAL EVALUATION

In [43]:
data = {
    "Dataset_Name": [dataset_name] * 6,
    "Number of Row": [df.shape[0]] * 6,  
    "Precision": [precision] * 6,
    "Degree": [degree] * 6,
    "Number_Of_Features": [d] * 6,
    "Model": ["Neal", "Neal", "Fixstar", "Fixstar", "Sklearn", "Sklearn"],
    "Dataset_split": ["Train", "Test", "Train", "Test", "Train", "Test"],
    "MSE": [
        train_mse_neal, test_mse_neal,
        train_mse_fixstar, test_mse_fixstar,
        train_mse_sklearn, test_mse_sklearn
    ],
    "MAE": [
        train_mae_neal, test_mae_neal,
        train_mae_fixstar, test_mae_fixstar,
        train_mae_sklearn, test_mae_sklearn
    ],
    "R2 Score": [
        train_r2_neal, test_r2_neal,
        train_r2_fixstar, test_r2_fixstar,
        train_r2_sklearn, test_r2_sklearn
    ],
    "preproccesing_time": [
        neal_preproccesing_time, neal_preproccesing_time,
        fixstar_preproccesing_time, fixstar_preproccesing_time,
        "-", "-" 
    ],
    "execution_time": [
        neal_execution_time, neal_execution_time,
        fixstar_execution_time, fixstar_execution_time,
        sklearn_execution_time, sklearn_execution_time
    ],
    "postproccesing_time": [
        neal_postproccesing_time, neal_postproccesing_time,
        fixstar_postproccesing_time, fixstar_postproccesing_time,
        "-", "-"
    ],
    "total_time": [
        neal_preproccesing_time + neal_execution_time + neal_postproccesing_time,neal_preproccesing_time + neal_execution_time + neal_postproccesing_time,
        fixstar_preproccesing_time + fixstar_execution_time + fixstar_postproccesing_time,fixstar_preproccesing_time + fixstar_execution_time + fixstar_postproccesing_time,
        sklearn_execution_time, sklearn_execution_time
    ]
}

df_eval = pd.DataFrame(data)
df_eval

,Dataset_Name,Number of Row,Precision,Degree,Number_Of_Features,Model,Dataset_split,MSE,MAE,R2 Score,preproccesing_time,execution_time,postproccesing_time,total_time
0,dataset_6.csv,36733,3,4,35,Neal,Train,0.041394,0.161274,-0.071740,0.063122,0.419859,0.010459,0.493441
1,dataset_6.csv,36733,3,4,35,Neal,Test,0.040970,0.159219,-0.061961,0.063122,0.419859,0.010459,0.493441
2,dataset_6.csv,36733,3,4,35,Fixstar,Train,0.035828,0.148573,0.072389,0.140224,5.825801,0.0,5.966025
3,dataset_6.csv,36733,3,4,35,Fixstar,Test,0.035734,0.147978,0.073754,0.140224,5.825801,0.0,5.966025
4,dataset_6.csv,36733,3,4,35,Sklearn,Train,0.035448,0.147472,0.082231,-,0.022553,-,0.022553
5,dataset_6.csv,36733,3,4,35,Sklearn,Test,0.035372,0.146933,0.083149,-,0.022553,-,0.022553


In [44]:
excel_filename = "evaluation_results2.xlsx"

# Add new data
append_to_excel_enhanced(
    excel_filename,
    df_eval,
    check_columns=['Dataset_Name', 'Precision', 'Degree']
)

Data successfully appended to Excel file.
